In [1]:
import tensorflow as tf
from mtcnn.mtcnn import MTCNN
import PIL
import numpy as np
from sklearn import preprocessing

In [2]:
model = tf.keras.models.load_model('facenet_keras.h5')

W1030 21:11:08.271795 4581832128 hdf5_format.py:221] No training configuration found in save file: the model was *not* compiled. Compile it manually.


In [3]:
for layer in model.layers:
    layer.trainable=False
dense1 = tf.keras.layers.Dense(64)(model.outputs[0])
out = tf.keras.layers.Dense(2, activation='softmax')(dense1)
model = tf.keras.Model(model.inputs[0], out)
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["acc"])

W1030 21:11:08.308647 4581832128 deprecation.py:506] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [17]:
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>]


In [5]:
print(model.inputs)
print(model.outputs)

[<tf.Tensor 'input_1:0' shape=(?, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'dense_1/Softmax:0' shape=(?, 2) dtype=float32>]


In [6]:
def extract_face(img_loc):
    image = PIL.Image.open(img_loc)
    image = image.convert('RGB')
    pixels = np.asarray(image)
    detector = MTCNN()
    results = detector.detect_faces(pixels)
    x1, y1, width, height = results[0]['box']
    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    face = pixels[y1:y2, x1:x2]
    image = PIL.Image.fromarray(face)
    image = image.resize((160, 160))
    face_array = np.asarray(image)
    mean, std = face_array.mean(), face_array.std()
    face_array = (face_array - mean) / std
    return face_array

In [7]:
from os import listdir
from os.path import isfile, join
dir1 = "/Users/zetong/Faces/Qi"
dir2 = "/Users/zetong/Faces/Pitt"
qi = [join(dir1, f) for f in listdir(dir1) if isfile(join(dir1, f))]
pitt = [join(dir2, f) for f in listdir(dir2) if isfile(join(dir2, f))]
qi_y = [0]*len(qi)
pitt_y = [1]*len(pitt)
y = qi_y + pitt_y
X = qi + pitt
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

## Real time image processing pipeline (saves memory with high time overhead)

In [2]:
def make_dataset(xarr, yarr):
    dataset = tf.data.Dataset.from_tensor_slices((xarr, yarr)).repeat()
    
    def _extract_face(img_loc):
        img_loc = img_loc.numpy().decode()
        image = PIL.Image.open(img_loc)
        image = image.convert('RGB')
        pixels = np.asarray(image)
        detector = MTCNN()
        results = detector.detect_faces(pixels)
        x1, y1, width, height = results[0]['box']
        x1, y1 = abs(x1), abs(y1)
        x2, y2 = x1 + width, y1 + height
        face = pixels[y1:y2, x1:x2]
        image = PIL.Image.fromarray(face)
        image = image.resize((160, 160))
        face_array = np.asarray(image)
        mean, std = face_array.mean(), face_array.std()
        face_array = (face_array - mean) / std
        return face_array
    
    def _process_datapair(X, y):
        X = tf.py_function(_extract_face, [X], tf.float32)
        X.set_shape([160, 160, 3])
        y.set_shape([])
        return X, y
    dataset = dataset.map(_process_datapair)
    return dataset.shuffle(buffer_size=200).batch(8).prefetch(8)

In [156]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1, random_state=42)

In [157]:
train = make_dataset(X_train, y_train)
val = make_dataset(X_val, y_val)
test = make_dataset(X_test, y_test)

In [ ]:
batch_size = 8
filename = "QiPittFace.h5"
model.fit(train, validation_data=val, validation_steps = len(X_val) / batch_size, steps_per_epoch=len(X_train) / batch_size, epochs=20)

Epoch 1/20


## Preprocess all images to tensors (fast training with high memory overhead)

In [8]:
X_arr = np.array([extract_face(x) for x in X])
np.save("train_data.npy", X_arr)

W1030 21:14:03.949308 4581832128 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mtcnn/mtcnn.py:187: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1030 21:14:03.950679 4581832128 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mtcnn/mtcnn.py:193: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1030 21:14:03.972729 4581832128 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mtcnn/network.py:43: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W1030 21:14:03.973877 4581832128 deprecation_wrapper.py:119] From /Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/mtcnn/layer_factory.py:88: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_arr, y, test_size=0.2, random_state=42)

In [10]:
model.fit(x=X_train, y=y_train, validation_split=0.1, batch_size=8, epochs=30)

Train on 155 samples, validate on 18 samples
Epoch 1/30
155/155 [==============================] - 7s 47ms/sample - loss: 0.4744 - acc: 0.8129 - val_loss: 0.1001 - val_acc: 0.9444
Epoch 2/30
155/155 [==============================] - 5s 32ms/sample - loss: 0.3329 - acc: 0.8452 - val_loss: 0.1842 - val_acc: 0.9444
Epoch 3/30
155/155 [==============================] - 5s 32ms/sample - loss: 0.2899 - acc: 0.8710 - val_loss: 0.1963 - val_acc: 0.9444
Epoch 4/30
155/155 [==============================] - 5s 32ms/sample - loss: 0.2180 - acc: 0.9097 - val_loss: 0.1305 - val_acc: 1.0000
Epoch 5/30
155/155 [==============================] - 5s 33ms/sample - loss: 0.1621 - acc: 0.9548 - val_loss: 0.0758 - val_acc: 1.0000
Epoch 6/30
155/155 [==============================] - 5s 33ms/sample - loss: 0.1177 - acc: 0.9742 - val_loss: 0.0310 - val_acc: 1.0000
Epoch 7/30
155/155 [==============================] - 5s 32ms/sample - loss: 0.0870 - acc: 0.9677 - val_loss: 0.0348 - val_acc: 1.0000
Epoch 8/30

In [14]:
model.evaluate(x=X_test, y=y_test, batch_size=8)

44/44 [==============================] - 1s 31ms/sample - loss: 0.1104 - acc: 0.9545


[0.11038615647703409, 0.95454544]

In [16]:
model.predict(np.expand_dims(extract_face("/Users/zetong/Desktop/qi2.jpg"), axis=0))

array([[0.67124164, 0.3287584 ]], dtype=float32)